<a href="https://colab.research.google.com/github/ExploitSori/2017Codegate_Drive-ByDownload/blob/master/%EB%B9%85%EC%9D%91%EB%B3%B4_case3_4_AE_IF_LOF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from google.colab import drive
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense
drive.mount('/content/drive')




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [45]:

data = pd.read_csv('./drive/MyDrive/Colab Notebooks/team_prj/data/type3_normal.csv')#.to_numpy()
y = data['type']
y = y.map({'Normal': 0, 'Attack': 1}).to_numpy()
#X['messageID'] = data['messageID']
X = data.drop(columns =['type'])
X['messageID'] = X['messageID'].astype('category')
#X = X.drop(columns =['isCan0','isService','serviceNODEID'])
X = X.drop(columns = ['time','data[0]','data[7]'])
print(X.shape)


(101601, 14)


In [46]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
input_dim = X_scaled.shape[1]
encoding_dim = 14

input_layer = Input(shape=(input_dim,))
encoder = Dense(encoding_dim, activation="relu")(input_layer)
decoder = Dense(input_dim, activation="sigmoid")(encoder)

autoencoder = Model(inputs=input_layer, outputs=decoder)
autoencoder.compile(optimizer='adam', loss='mse')

# Autoencoder 학습
autoencoder.fit(X_scaled, X_scaled, epochs=30, batch_size=32, shuffle=True, validation_split=0.2)

Epoch 1/30
2540/2540 [==============================] - 6s 2ms/step - loss: 0.6365 - val_loss: 0.5251
Epoch 2/30
2540/2540 [==============================] - 6s 2ms/step - loss: 0.5740 - val_loss: 0.5181
Epoch 3/30
2540/2540 [==============================] - 5s 2ms/step - loss: 0.5697 - val_loss: 0.5148
Epoch 4/30
2540/2540 [==============================] - 6s 2ms/step - loss: 0.5691 - val_loss: 0.5142
Epoch 5/30
2540/2540 [==============================] - 5s 2ms/step - loss: 0.5690 - val_loss: 0.5140
Epoch 6/30
2540/2540 [==============================] - 5s 2ms/step - loss: 0.5689 - val_loss: 0.5140
Epoch 7/30
2540/2540 [==============================] - 6s 2ms/step - loss: 0.5683 - val_loss: 0.5133
Epoch 8/30
2540/2540 [==============================] - 6s 2ms/step - loss: 0.5681 - val_loss: 0.5132
Epoch 9/30
2540/2540 [==============================] - 6s 2ms/step - loss: 0.5681 - val_loss: 0.5131
Epoch 10/30
2540/2540 [==============================] - 5s 2ms/step - loss: 0.568

In [47]:
input_layer = Input(shape=(input_dim,))
encoder = Dense(encoding_dim, activation="relu")(input_layer)
decoder = Dense(input_dim, activation="sigmoid")(encoder)
encoder_model = Model(inputs=input_layer, outputs=encoder)

# 데이터를 인코딩
X_encoded = encoder_model.predict(X_scaled)

3176/3176 [==============================] - 3s 1ms/step


In [48]:
#data.loc[data['type']=='Normal', 'type'] = 0
#data.loc[data['type']=='Attack', 'type'] = 1

fullDataX1 = pd.read_csv('./drive/MyDrive/Colab Notebooks/team_prj/data/type3.csv')#.to_numpy()

#X['messageID'] = data['messageID']
fX1 = fullDataX1.drop(columns =['type'])
#fX1 = fX1.drop(columns =['isCan0','isService','serviceNODEID'])
fX1 = fX1.drop(columns = ['time','data[0]','data[7]'])
print(fX1.shape)
fX1_scaled = scaler.fit_transform(fX1)
fX1_encoded = encoder_model.predict(fX1_scaled)

(197479, 14)
6172/6172 [==============================] - 6s 1ms/step


In [49]:
if_model = IsolationForest(contamination=0.1)  # contamination 값 조정 가능
if_model.fit(X_encoded)

# 이상치 예측
if_predictions = if_model.predict(fX1_encoded)

In [57]:

print(np.unique(if_predictions, return_counts=True))

(array([-1,  1]), array([ 84878, 112601]))


In [60]:
# 모델 저장
#import pickle
#with open('/content/drive/MyDrive/Colab Notebooks/team_prj/case3_if_model.pkl', 'wb') as file:
#    pickle.dump(if_model, file)

In [66]:
import pickle
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report
import pickle
with open('/content/drive/MyDrive/Colab Notebooks/team_prj/case3_if_model.pkl', 'rb') as file:
    loaded_if_model = pickle.load(file)

# 불러온 모델로 예측
fy1 = fullDataX1['type']
fy1 = fy1.map({'Normal': 0, 'Attack': 1}).to_numpy()
loaded_if_predictions = loaded_if_model.predict(fX1_encoded)
loaded_if_predictions = np.where(loaded_if_predictions == -1, 0, loaded_if_predictions)
print(np.unique(loaded_if_predictions, return_counts=True))
conf_matrix = confusion_matrix(fy1, loaded_if_predictions)
class_report = classification_report(fy1, loaded_if_predictions)

print("Confusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(class_report)

(array([0, 1]), array([ 84878, 112601]))
Confusion Matrix:
[[54118 47483]
 [30760 65118]]

Classification Report:
              precision    recall  f1-score   support

           0       0.64      0.53      0.58    101601
           1       0.58      0.68      0.62     95878

    accuracy                           0.60    197479
   macro avg       0.61      0.61      0.60    197479
weighted avg       0.61      0.60      0.60    197479



In [50]:
# Local Outlier Factor 모델 정의 및 학습
lof_model = LocalOutlierFactor(n_neighbors=30, contamination=0.1, novelty=True)  # n_neighbors 및 contamination 값 조정 가능
lof_model.fit(X_encoded)
lof_predictions = lof_model.predict(fX1_encoded)
# 이상치라면 -1, 정상치라면 1로 표시됨


In [55]:
# 모델 저장
import pickle
with open('/content/drive/MyDrive/Colab Notebooks/team_prj/case3_lof_model.pkl', 'wb') as file:
    pickle.dump(lof_model, file)

In [59]:
with open('/content/drive/MyDrive/Colab Notebooks/team_prj/case3_lof_model.pkl', 'rb') as file:
    loaded_lof_model = pickle.load(file)

# 불러온 모델로 예측
loaded_lof_predictions = loaded_lof_model.predict(fX1_encoded)
print(np.unique(loaded_lof_predictions, return_counts=True))

In [58]:
print(np.unique(lof_predictions, return_counts=True))

(array([-1,  1]), array([196519,    960]))


In [53]:
data2 = pd.read_csv('./drive/MyDrive/Colab Notebooks/team_prj/data/type2.csv')
y2 = data2['type']
y2 = y2.map({'Normal': 0, 'Attack': 1}).to_numpy()
#X2 = data2.drop(columns =['type'])
#X2 = X2.drop(columns =['isCan0'])
#X2 = X2.drop(columns = ['time'])

X2 = data2.drop(columns =['type'])
X2 = X2.drop(columns =['isCan0','isService','serviceNODEID'])
X2 = X2.drop(columns = ['time','data[0]','data[7]'])

